https://www.standardmedia.co.ke/sci-tech/article/2001386004/ai-robot-develops-the-perfect-tiktok-song-and-it-s-very-unusual

# Setup

In [1]:
import os
import re
import requests
from time import sleep

import numpy as np
import pandas as pd
import spotipy
from bs4 import BeautifulSoup
from dotenv import load_dotenv, find_dotenv
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth

# Web Scrapping

In [2]:
url = 'https://www.buzzfeednews.com/article/laurenstrapagiel/most-viral-tiktok-songs-of-2020'

In [3]:
response = requests.get(url)
response

<Response [200]>

In [4]:
content_buzzfeed = BeautifulSoup(response.text)
#content

In [5]:
html_buzzfeed = content_buzzfeed.find_all('h2', limit=13)
html_buzzfeed

[<h2>1. "Laxed (SIREN BEAT)" by Jawsh 685</h2>,
 <h2>2. "Roses (Imanbek Remix)" by SAINt JHN and Imanbek</h2>,
 <h2>3. "Boss Bitch" by Doja Cat</h2>,
 <h2>4. "Supalonely" by BENEE feat. Gus Dapperton</h2>,
 <h2>5. "Stunnin'" by Curtis Waters feat. Harm Franklin</h2>,
 <h2>6. "Lottery'" by K CAMP</h2>,
 <h2>7. "ROXANNE'" by Arizona Zervas</h2>,
 <h2>8. "Bored in the House'" by Curtis Roach</h2>,
 <h2>9. "Sober Up" by AJR feat. Rivers Cuomo</h2>,
 <h2>10. "Prom Queen" by Beach Bunny</h2>,
 <h2>11. "Vibe" by Cookiee Kawaii</h2>,
 <h2>12. "Renee" by SALES</h2>,
 <h2>13. "Savage" by Megan Thee Stallion</h2>]

In [6]:
date_buzzfeed = content_buzzfeed.find('p', attrs={'class': 'news-article-header__timestamps-posted'}).text.replace('\n', '').strip()
date_buzzfeed

'Posted on July 23, 2020, at 3:24 p.m. ET'

In [7]:
url = 'https://www.popsugar.com/entertainment/popular-tiktok-songs-47289804?stream_view=1#photo-47289832'

In [8]:
response = requests.get(url)
response

<Response [200]>

In [9]:
content_popsugar = BeautifulSoup(response.text)
#content_popsugar

In [10]:
html_popsugar = content_popsugar.find_all('span', attrs={'class': 'count-copy'})
#html_popsugar

In [11]:
date_popsugar = content_popsugar.find('time').text.replace('\n', '').strip()
date_popsugar

'March 27, 2020'

In [12]:
popsugar_df = pd.DataFrame([re.split(' by | feat. ', song.text.replace('"', '').strip()) for song in html_popsugar], 
                             columns=['song', 'artist', 'feat'])
popsugar_df

,song,artist,feat
0,Roxanne,Arizona Zervas,None
1,Say So,Doja Cat,None
2,My Oh My,Camila Cabello,DaBaby
3,Moon,Kid Francescoli,None
4,Vibe,Cookiee Kawaii,None
...,...,...,...
64,What the Hell,Avril Lavigne,None
65,Towards the Sun,Rihanna,None
66,I Think I'm OKAY,"Machine Gun Kelly, YUNGBLUD, and Travis Barker",None
67,Myself,Bazzi,None


In [13]:
popsugar_df_raw = popsugar_df.copy()

# Spotify

https://spotipy.readthedocs.io/en/2.14.0/#module-spotipy.client

https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/

## Connecting to the API

In [14]:
load_dotenv(find_dotenv())

True

In [15]:
cid = os.getenv('spotify_p03_key')
csecret = os.getenv('spotify_p03_secret')

In [16]:
cc_manager = SpotifyClientCredentials(client_id=cid, client_secret=csecret)
sp = spotipy.Spotify(client_credentials_manager=cc_manager)

In [17]:
# Search in the api wrapper
spotify_search = [sp.search(q=song, type='track') for song in popsugar_df['song']]

In [18]:
popsugar_df['spotify_id'] = [track['tracks']['items'][0]['id'] if len(track['tracks']['items']) != 0 else 'not-found' 
                             for track in spotify_search]

In [19]:
popsugar_df['popularity'] = [track['tracks']['items'][0]['popularity'] if len(track['tracks']['items']) != 0 
                             else 'not-found' for track in spotify_search]

In [20]:
popsugar_df['release_date'] = [track['tracks']['items'][0]['album']['release_date'] if len(track['tracks']['items']) != 0 
                               else 'not-found' for track in spotify_search]

In [21]:
popsugar_df['explicit'] = [track['tracks']['items'][0]['explicit'] if len(track['tracks']['items']) != 0 
                           else 'not-found' for track in spotify_search]

In [22]:
popsugar_df.head()

,song,artist,feat,spotify_id,popularity,release_date,explicit
0,Roxanne,Arizona Zervas,None,696DnlkuDOXcMAnKlTgXXK,89,2019-10-10,True
1,Say So,Doja Cat,None,3Dv1eDb0MEgF93GpLXlucZ,89,2019-11-07,True
2,My Oh My,Camila Cabello,DaBaby,3yOlyBJuViE2YSGn3nVE1K,83,2019-12-06,False
3,Moon,Kid Francescoli,None,0JP9xo3adEtGSdUEISiszL,86,2018-03-16,True
4,Vibe,Cookiee Kawaii,None,0fySG6A6qLE8IvDpayb5bM,80,2019-09-27,True


In [23]:
spotipy_au_feat = [sp.audio_features(id) if id != 'not-found' else 'not-found' for id in popsugar_df.spotify_id]

In [24]:
spotipy_au_feat[0][0]

{'danceability': 0.621,
 'energy': 0.601,
 'key': 6,
 'loudness': -5.616,
 'mode': 0,
 'speechiness': 0.148,
 'acousticness': 0.0522,
 'instrumentalness': 0,
 'liveness': 0.46,
 'valence': 0.457,
 'tempo': 116.735,
 'type': 'audio_features',
 'id': '696DnlkuDOXcMAnKlTgXXK',
 'uri': 'spotify:track:696DnlkuDOXcMAnKlTgXXK',
 'track_href': 'https://api.spotify.com/v1/tracks/696DnlkuDOXcMAnKlTgXXK',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/696DnlkuDOXcMAnKlTgXXK',
 'duration_ms': 163636,
 'time_signature': 5}

In [25]:
spotipy_au_feat_keys = list(spotipy_au_feat[0][0].keys())
spotipy_au_feat_keys.remove('type')
spotipy_au_feat_keys.remove('id')
spotipy_au_feat_keys.remove('uri')
spotipy_au_feat_keys.remove('track_href')
spotipy_au_feat_keys.remove('analysis_url')
spotipy_au_feat_keys

['danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'duration_ms',
 'time_signature']

In [26]:
for key in spotipy_au_feat_keys:
    popsugar_df[key] = [feat[0][key] if feat != 'not-found' else 'not-found' for feat in spotipy_au_feat]

In [27]:
popsugar_df.head()

,song,artist,feat,spotify_id,popularity,release_date,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Roxanne,Arizona Zervas,None,696DnlkuDOXcMAnKlTgXXK,89,2019-10-10,True,0.621,0.601,6,-5.616,0,0.148,0.0522,0,0.46,0.457,116.735,163636,5
1,Say So,Doja Cat,None,3Dv1eDb0MEgF93GpLXlucZ,89,2019-11-07,True,0.787,0.673,11,-4.577,0,0.158,0.256,3.57e-06,0.0904,0.786,110.962,237893,4
2,My Oh My,Camila Cabello,DaBaby,3yOlyBJuViE2YSGn3nVE1K,83,2019-12-06,False,0.724,0.491,8,-6.024,1,0.0296,0.018,1.29e-05,0.0887,0.383,105.046,170746,4
3,Moon,Kid Francescoli,None,0JP9xo3adEtGSdUEISiszL,86,2018-03-16,True,0.921,0.537,9,-5.723,0,0.0804,0.556,0.00404,0.102,0.711,128.009,135090,4
4,Vibe,Cookiee Kawaii,None,0fySG6A6qLE8IvDpayb5bM,80,2019-09-27,True,0.768,0.652,1,-2.708,0,0.307,0.113,0,0.107,0.777,154.187,144935,4


### Track example

```python
[In]:
sp.search(q='My Oh My', type='track', limit=1)

[Out]:
{'tracks': {'href': 'https://api.spotify.com/v1/search?query=My+Oh+My&type=track&offset=0&limit=1',
            # Track info
            # Album info
            'items': [{'album': {'album_type': 'album',
                                 # Main artist info (about the album)
                                 'artists': [{'external_urls': 
                                              {'spotify': 'https://open.spotify.com/artist/4nDoRrQiYLoBzwC5BhVJzF'},
                                              'href': 'https://api.spotify.com/v1/artists/4nDoRrQiYLoBzwC5BhVJzF',
                                              # Artist id
                                              'id': '4nDoRrQiYLoBzwC5BhVJzF',
                                              # Artist name
                                              'name': 'Camila Cabello',
                                              'type': 'artist',
                                              'uri': 'spotify:artist:4nDoRrQiYLoBzwC5BhVJzF'}],
                                 # Where the album is available
                                 'available_markets': ['AD', 'AE', 'AL', 'AR', 'AT', 'AU', 'BA', 'BE', 'BG', 'BH', 'BO',
                                                       'BR', 'BY', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DE', 'DK',
                                                       'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FR', 'GB', 'GR', 'GT',
                                                       'HK', 'HN', 'HR', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS', 'IT', 'JO',
                                                       'JP', 'KW', 'KZ', 'LB', 'LI', 'LT', 'LU', 'LV', 'MA', 'MC', 'MD',
                                                       'ME', 'MK', 'MT', 'MX', 'MY', 'NI', 'NL', 'NO', 'NZ', 'OM', 'PA',
                                                       'PE', 'PH', 'PL', 'PS', 'PT', 'PY', 'QA', 'RO', 'RS', 'RU', 'SA',
                                                       'SE', 'SG', 'SI', 'SK', 'SV', 'TH', 'TN', 'TR', 'TW', 'UA', 'US',
                                                       'UY', 'VN', 'XK', 'ZA'],
                                 'external_urls': {'spotify': 'https://open.spotify.com/album/3Vsbl0diFGw8HNSjG8ue9m'},
                                 'href': 'https://api.spotify.com/v1/albums/3Vsbl0diFGw8HNSjG8ue9m',
                                 # Album id
                                 'id': '3Vsbl0diFGw8HNSjG8ue9m',
                                 # Album images
                                 'images': [{'height': 640,
                                             'url': 'https://i.scdn.co/image/ab67616d0000b2735f53c0dbe5190a0af0fa28f3',
                                             'width': 640},
                                            {'height': 300,
                                             'url': 'https://i.scdn.co/image/ab67616d00001e025f53c0dbe5190a0af0fa28f3',
                                             'width': 300},
                                            {'height': 64,
                                             'url': 'https://i.scdn.co/image/ab67616d000048515f53c0dbe5190a0af0fa28f3',
                                             'width': 64}],
                                 # Album name
                                 'name': 'Romance',
                                 # Album release date
                                 'release_date': '2019-12-06',
                                 'release_date_precision': 'day',
                                 # Number of tracks in the album
                                 'total_tracks': 14,
                                 'type': 'album',
                                 'uri': 'spotify:album:3Vsbl0diFGw8HNSjG8ue9m'},
                       # Artists info (about the track)
                       'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4nDoRrQiYLoBzwC5BhVJzF'},
                                    'href': 'https://api.spotify.com/v1/artists/4nDoRrQiYLoBzwC5BhVJzF',
                                    'id': '4nDoRrQiYLoBzwC5BhVJzF',
                                    'name': 'Camila Cabello',
                                    'type': 'artist',
                                    'uri': 'spotify:artist:4nDoRrQiYLoBzwC5BhVJzF'},
                                   # Feat
                                   {'external_urls': {'spotify': 'https://open.spotify.com/artist/4r63FhuTkUYltbVAg5TQnk'},
                                    'href': 'https://api.spotify.com/v1/artists/4r63FhuTkUYltbVAg5TQnk',
                                    'id': '4r63FhuTkUYltbVAg5TQnk',
                                    'name': 'DaBaby',
                                    'type': 'artist',
                                    'uri': 'spotify:artist:4r63FhuTkUYltbVAg5TQnk'}],
                       # Where the track is available
                       'available_markets': ['AD', 'AE', 'AL', 'AR', 'AT', 'AU', 'BA', 'BE', 'BG', 'BH', 'BO', 'BR', 'BY',
                                             'CA', 'CH', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DE', 'DK', 'DO', 'DZ', 'EC', 'EE',
                                             'EG', 'ES', 'FI', 'FR', 'GB', 'GR', 'GT', 'HK', 'HN', 'HR', 'HU', 'ID', 'IE',
                                             'IL', 'IN', 'IS', 'IT', 'JO', 'JP', 'KW', 'KZ', 'LB', 'LI', 'LT', 'LU', 'LV',
                                             'MA', 'MC', 'MD', 'ME', 'MK', 'MT', 'MX', 'MY', 'NI', 'NL', 'NO', 'NZ', 'OM',
                                             'PA', 'PE', 'PH', 'PL', 'PS', 'PT', 'PY', 'QA', 'RO', 'RS', 'RU', 'SA', 'SE',
                                             'SG', 'SI', 'SK', 'SV', 'TH', 'TN', 'TR', 'TW', 'UA', 'US', 'UY', 'VN', 'XK',
                                             'ZA'],
                       'disc_number': 1,
                       # Durantion of the track (ms)
                       'duration_ms': 170746,
                       # Explicit?
                       'explicit': False,
                       'external_ids': {'isrc': 'USSM11914257'},
                       'external_urls': {'spotify': 'https://open.spotify.com/track/3yOlyBJuViE2YSGn3nVE1K'},
                       'href': 'https://api.spotify.com/v1/tracks/3yOlyBJuViE2YSGn3nVE1K',
                       # Track id
                       'id': '3yOlyBJuViE2YSGn3nVE1K',
                       'is_local': False,
                       # Track name
                       'name': 'My Oh My (feat. DaBaby)',
                       # Track popularity
                       'popularity': 83,
                       'preview_url': 'https://p.scdn.co/mp3-preview/f4d0ff165605108542de7358547443d1a9f33fe3?cid=52d383b8c23d4cf987d4a0d509588d7a',
                       'track_number': 4,
                       'type': 'track',
                       'uri': 'spotify:track:3yOlyBJuViE2YSGn3nVE1K'}],
            'limit': 1,
            'next': 'https://api.spotify.com/v1/search?query=My+Oh+My&type=track&offset=1&limit=1',
            'offset': 0,
            'previous': None,
            'total': 25565}}```

### Artist example

```python
[In]:
sp.search(q='Camila Cabello', type='artist', limit=1)

[Out]:
# Artist info
{'artists': {'href': 'https://api.spotify.com/v1/search?query=Camila+Cabello&type=artist&offset=0&limit=1',
             'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4nDoRrQiYLoBzwC5BhVJzF'},
                        # Followers
                        'followers': {'href': None, 'total': 17921890},
                        # Genres
                        'genres': ['dance pop', 'pop', 'post-teen pop'],
                        'href': 'https://api.spotify.com/v1/artists/4nDoRrQiYLoBzwC5BhVJzF',
                        # Artist id
                        'id': '4nDoRrQiYLoBzwC5BhVJzF',
                        'images': [{'height': 640,
                                    'url': 'https://i.scdn.co/image/667daebca4856db3f4cbe8f51a4881091e62c401',
                                    'width': 640},
                                   {'height': 320,
                                    'url': 'https://i.scdn.co/image/6628c8b4acfa56206eaf83edd561f0549f8f23df',
                                    'width': 320},
                                   {'height': 160,
                                    'url': 'https://i.scdn.co/image/9beb36d55d0a4784f4adf06fb506d292c688b8bf',
                                    'width': 160}],
                        # Artist name
                        'name': 'Camila Cabello',
                        # Artist popularity
                        'popularity': 87,
                        'type': 'artist',
                        'uri': 'spotify:artist:4nDoRrQiYLoBzwC5BhVJzF'}],
             'limit': 1,
             'next': 'https://api.spotify.com/v1/search?query=Camila+Cabello&type=artist&offset=1&limit=1',
             'offset': 0,
             'previous': None,
             'total': 3}}```

### Audio features

```python
[In]:
sp.audio_features('3yOlyBJuViE2YSGn3nVE1K')

[Out]:
[{'danceability': 0.724,
  'energy': 0.491,
  'key': 8,
  'loudness': -6.024,
  'mode': 1,
  'speechiness': 0.0296,
  'acousticness': 0.018,
  'instrumentalness': 1.29e-05,
  'liveness': 0.0887,
  'valence': 0.383,
  'tempo': 105.046,
  'type': 'audio_features',
  'id': '3yOlyBJuViE2YSGn3nVE1K',
  'uri': 'spotify:track:3yOlyBJuViE2YSGn3nVE1K',
  'track_href': 'https://api.spotify.com/v1/tracks/3yOlyBJuViE2YSGn3nVE1K',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3yOlyBJuViE2YSGn3nVE1K',
  'duration_ms': 170746,
  'time_signature': 4}]```

# Last.fm

https://www.last.fm/api/show/track.getInfo



headers = {'user-agent': lastfm_user}

payload = {
    'api_key': lastfm_key,
    'method': 'track.getInfo'
}

response = requests.get('http://ws.audioscrobbler.com/2.0/', headers=headers, params=payload)
response

In [28]:
lastfm_key = os.getenv('lastfm_p03_key')
lastfm_user = 'gnakasato'

In [29]:
def lastfm_get(payload):
    # Define the headers and the url
    headers = {'user-agent': lastfm_user}
    url = 'http://ws.audioscrobbler.com/2.0/'
    
    # Add API key and format to the payload
    payload['api_key'] = lastfm_key
    payload['format=json'] = lastfm_key
    
    response = requests.get(url, headers=headers, params=payload)
    
    return response

In [88]:
response = lastfm_get({
    'method': 'track.getInfo',
    'track': 'My Oh My',
    'artist': 'Camila Cabello'
})

In [89]:
response

<Response [200]>

In [30]:
lastfm_track_info = []

for index in range(popsugar_df.shape[0]):
    response = lastfm_get({'method': 'track.getInfo',
                           'track': popsugar_df.iloc[index, 0],
                           'artist': popsugar_df.iloc[index, 1]})
    track_info = BeautifulSoup(response.text)
    lastfm_track_info.append(track_info)
    sleep(1)

In [87]:
lastfm_tags = [track.find_all('tag') if track.text.replace('\n\n', '') != 'Track not found' else 'not-found' for track 
                in lastfm_track_info]

In [89]:
tracks_tags = []

for track_tag in lastfm_tags:
    if track_tag == 'not-found':
        tracks_tags.append(['not-found'])
        #print(tracks_tags)
    elif len(track_tag) == 0:
        tracks_tags.append(['no-tag'])
        #print(tracks_tags)
    else:
        each_tag_lists = []
        for each_tag in track_tag:
            
            tag_list = each_tag.find('name')
            each_tag_lists.append(tag_list)
            #print(f'each_tag_lists: {each_tag_lists}')
            each_track_tags = []
            for each_one in each_tag_lists:
                tag = each_one.text
                each_track_tags.append(tag)
                #print(f'each_track_tags: {each_track_tags}')
        tracks_tags.append(each_track_tags)
        #print(f'\n{tracks_tags}\n')
        
popsugar_df['lastfm_tags'] = tracks_tags

In [90]:
popsugar_df.head()

,song,artist,feat,spotify_id,popularity,release_date,explicit,danceability,energy,key,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,lastfm_tags
0,Roxanne,Arizona Zervas,None,696DnlkuDOXcMAnKlTgXXK,89,2019-10-10,True,0.621,0.601,6,...,0,0.148,0.0522,0,0.46,0.457,116.735,163636,5,"[2019, 2010s, arizona zervas, Hip-Hop, rap]"
1,Say So,Doja Cat,None,3Dv1eDb0MEgF93GpLXlucZ,89,2019-11-07,True,0.787,0.673,11,...,0,0.158,0.256,3.57e-06,0.0904,0.786,110.962,237893,4,"[pop, Disco, rap, Hip-Hop, female vocalists]"
2,My Oh My,Camila Cabello,DaBaby,3yOlyBJuViE2YSGn3nVE1K,83,2019-12-06,False,0.724,0.491,8,...,1,0.0296,0.018,1.29e-05,0.0887,0.383,105.046,170746,4,"[pop, fuck feat tags, DaBaby]"
3,Moon,Kid Francescoli,None,0JP9xo3adEtGSdUEISiszL,86,2018-03-16,True,0.921,0.537,9,...,0,0.0804,0.556,0.00404,0.102,0.711,128.009,135090,4,"[chillout, indie pop, skam italia]"
4,Vibe,Cookiee Kawaii,None,0fySG6A6qLE8IvDpayb5bM,80,2019-09-27,True,0.768,0.652,1,...,0,0.307,0.113,0,0.107,0.777,154.187,144935,4,[no-tag]
